In [1]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from scipy import signal

import os
from glob import glob
import matplotlib.pyplot as plt


Folder Paths

In [2]:
greenPath = './ppg_GREEN/'
redPath = './ppg_RED/'
irPath = './ppg_IR/'

BandPass Filter

In [6]:
# BandPass Filter Parameters

sampling_freq = 100 # Hz

lowcut = 0.5                         # Lower Cutoff Frequency in Hz
highcut = 35.0                       # Higher Cutoff Frequency in Hz
nyquist_freq = 0.5 * sampling_freq   # Nyquist frequency in Hz
order = 4                            # Filter order

# Design Bandpass Filter
b, a = signal.butter(order, [lowcut / nyquist_freq, highcut / nyquist_freq], btype='band')

BandPass Filter : AC Component Calculation

In [7]:
# Average AC Component of Trials
avgAC = 0
numOfFiles = 0

print("BANDPASS\n")

for file in glob(greenPath + '*.csv'):
    numOfFiles += 1
    ppg_data = pd.read_csv(file)
    noisy_ppgOne = ppg_data["PPG1"].values
    # Apply BandPass Filter to noisy ECG Signal
    filtered_ppg_1 = signal.filtfilt(b, a, noisy_ppgOne)

    # Baseline Correction
    baseline_ppg_1 = signal.medfilt(filtered_ppg_1, kernel_size=51)  # Apply a median filter as a baseline
    corrected_ppg_1 = filtered_ppg_1 - baseline_ppg_1

    # Smoothing
    smoothed_ppg_1 = signal.savgol_filter(corrected_ppg_1, window_length=51, polyorder=3)

    # Remove the DC Component
    dc_component = np.mean(smoothed_ppg_1)
    ppg_ac= smoothed_ppg_1 - dc_component

    # Calculate the AC Component
    peaks, _ = find_peaks(ppg_ac)  # Adjust height threshold as needed
    troughs, _ = find_peaks(-ppg_ac)  # Detect troughs by inverting signal
    ac_component = ppg_ac[peaks].mean() - ppg_ac[troughs].mean()

    # Add to avg
    avgAC += ac_component

    # Print the AC Component for trial
    print(f'AC Component of {file} : {ac_component}')

print('---------------------------------------------------')
print(f'Average AC Component of Trials using Bandpass is : {avgAC / numOfFiles}')


BANDPASS

AC Component of ./ppg_GREEN\green1_AR.csv : 8.978241222511024


TypeError: unsupported operand type(s) for -: 'str' and 'str'

BandPass Filter : Perfusion Index Calculation

In [48]:
# Average Perfusion Index of Trials
avgPerfusion = 0
numOfFiles = 0

print("BANDPASS\n")

for file in glob(greenPath + '*.csv'):
    numOfFiles += 1
    ppg_data = pd.read_csv(file)
    noisy_ppgOne = ppg_data["PPG1"].values
    # Apply BandPass Filter to noisy ECG Signal
    filtered_ppg_1 = signal.filtfilt(b, a, noisy_ppgOne)

    # Baseline Correction
    baseline_ppg_1 = signal.medfilt(filtered_ppg_1, kernel_size=51)  # Apply a median filter as a baseline
    corrected_ppg_1 = filtered_ppg_1 - baseline_ppg_1

    # Smoothing
    smoothed_ppg_1 = signal.savgol_filter(corrected_ppg_1, window_length=51, polyorder=3)

    # Find peaks and troughs-------------------------------------------------------------------------------
    peaks, _ = find_peaks(smoothed_ppg_1)  # Adjust height threshold as needed
    troughs, _ = find_peaks(-smoothed_ppg_1)  # Detect troughs by inverting signal

    # Calculate Pulse Amplitude
    if len(peaks) > 0 and len(troughs) > 0:
        pulse_amplitude = smoothed_ppg_1[peaks].mean() - smoothed_ppg_1[troughs].mean()
    else:
        pulse_amplitude = np.nan

    # Calculate Baseline Amplitude
    baseline_amplitude = smoothed_ppg_1.mean()

    # Calculate Perfusion Index
    if not np.isnan(pulse_amplitude) and not np.isnan(baseline_amplitude):
        perfusion_index = (pulse_amplitude / baseline_amplitude) * 100
    else:
        perfusion_index = np.nan

    # Add to avg
    avgPerfusion += perfusion_index

    # Print the AC Component for trial
    print(f'Perfusion Index of {file} : {perfusion_index}')

print('---------------------------------------------------')
print(f'Average Perfusion Index of Trials using Bandpass is : {avgPerfusion / numOfFiles}')

Perfusion Index of ./ppg_GREEN\green1_AR.csv : 655.1065524190552
Perfusion Index of ./ppg_GREEN\green2_AR.csv : 394.56079478986703
Perfusion Index of ./ppg_GREEN\green3_AR.csv : 169.3737216465913
Perfusion Index of ./ppg_GREEN\green4_AR.csv : 619.9279339001287
Perfusion Index of ./ppg_GREEN\green5_AR.csv : 310.99419639327994
---------------------------------------------------
Average Perfusion Index of Trials using Bandpass is : 429.9926398297845


In [59]:
for file in glob(greenPath + '*.csv'):
    numOfFiles += 1
    ppg_data = pd.read_csv(file)
    noisy_ppg_1 = ppg_data["PPG1"].values
    # Apply BandPass Filter to noisy ECG Signal
    filtered_ppg_1 = signal.filtfilt(b, a, noisy_ppgOne)

    # Baseline Correction
    baseline_ppg_1 = signal.medfilt(filtered_ppg_1, kernel_size=51)  # Apply a median filter as a baseline
    corrected_ppg_1 = filtered_ppg_1 - baseline_ppg_1

    # Smoothing
    smoothed_ppg_1 = signal.savgol_filter(corrected_ppg_1, window_length=51, polyorder=3)
    # Create a figure with subplots
    #fig, axs = plt.subplots(3, 1, figsize=(12, 10))

    # PPG 1
    # Plot original and filtered signals
    # axs[0].plot(ppg_data["VAL"], noisy_ppg_1, color='blue', label='Original PPG1')
    # axs[0].set_title('BANDPASS Original - PPG1 Signal')
    # axs[0].set_xlabel('Val (s)')
    # axs[0].set_ylabel('Amplitude')
    # axs[0].grid(True)


    # axs[1].plot(ppg_data["VAL"], filtered_ppg_1, color='green', label='Filtered PPG1')
    # axs[1].set_xlabel('Val (s)')
    # axs[1].set_ylabel('Amplitude')
    # axs[1].set_title('BANDPASS filtered - PPG1 Signal')
    # axs[1].grid(True)

    # # Plot original and filtered signals
    # axs[2].plot(ppg_data["VAL"], smoothed_ppg_1, color='purple', label='Corrected & Smoothed PPG1')
    # axs[2].set_title('BANDPASS Corrected & Smoothed - PPG1 Signal')
    # axs[2].set_xlabel('Val (s)')
    # axs[2].set_ylabel('Amplitude')
    # axs[2].grid(True)

    # Show the plot
    # plt.tight_layout()  # Adjust subplots to prevent overlap
    # plt.show()


Highpass Filter

In [60]:
# HighPass Filter Parameters
sampling_freq = 100 # Hz

cutoff = 0.5                         # Cutoff Frequency in Hz
nyquist_freq = 0.5 * sampling_freq   # Nyquist frequency in Hz
order = 4                            # Filter order

# Design Bandpass Filter
b, a = signal.butter(order, cutoff / nyquist_freq, btype='high')

HighPass Filter : AC Component Calculation

In [61]:
# Average AC Component of Trials
avgAC = 0
numOfFiles = 0

print("HIGHPASS\n")

for file in glob(greenPath + '*.csv'):
    numOfFiles += 1
    ppg_data = pd.read_csv(file)
    noisy_ppgOne = ppg_data["PPG1"].values
    
    # Apply BandPass Filter to noisy ECG Signal
    filtered_ppg_1 = signal.filtfilt(b, a, noisy_ppgOne)

    # Baseline Correction
    baseline_ppg_1 = signal.medfilt(filtered_ppg_1, kernel_size=51)  # Apply a median filter as a baseline
    corrected_ppg_1 = filtered_ppg_1 - baseline_ppg_1

    # Smoothing
    smoothed_ppg_1 = signal.savgol_filter(corrected_ppg_1, window_length=51, polyorder=3)

    # Remove the DC Component
    dc_component = np.mean(smoothed_ppg_1)
    ppg_ac= smoothed_ppg_1 - dc_component

    # Calculate the AC Component
    peaks, _ = find_peaks(ppg_ac)  # Adjust height threshold as needed
    troughs, _ = find_peaks(-ppg_ac)  # Detect troughs by inverting signal
    ac_component = ppg_ac[peaks].mean() - ppg_ac[troughs].mean()

    # Add to avg
    avgAC += ac_component

    # Print the AC Component for trial
    print(f'AC Component of {file} : {ac_component}')

print('---------------------------------------------------')
print(f'Average AC Component of Trials using HighPass is : {avgAC / numOfFiles}')


HIGHPASS

AC Component of ./ppg_GREEN\green1_AR.csv : 7.733767869140594
AC Component of ./ppg_GREEN\green2_AR.csv : 19.87857940833772
AC Component of ./ppg_GREEN\green3_AR.csv : 21.372674476544386
AC Component of ./ppg_GREEN\green4_AR.csv : 55.17601392305688
AC Component of ./ppg_GREEN\green5_AR.csv : 45.3889045419254
---------------------------------------------------
Average AC Component of Trials using HighPass is : 29.909988043800997


HighPass Filter : Perfusion Index Calculation

In [62]:
# Average Perfusion Index of Trials
avgPerfusion = 0
numOfFiles = 0

print("HIGHPASS\n")

for file in glob(greenPath + '*.csv'):
    numOfFiles += 1
    ppg_data = pd.read_csv(file)
    noisy_ppgOne = ppg_data["PPG1"].values
    # Apply BandPass Filter to noisy ECG Signal
    filtered_ppg_1 = signal.filtfilt(b, a, noisy_ppgOne)

    # Baseline Correction
    baseline_ppg_1 = signal.medfilt(filtered_ppg_1, kernel_size=51)  # Apply a median filter as a baseline
    corrected_ppg_1 = filtered_ppg_1 - baseline_ppg_1

    # Smoothing
    smoothed_ppg_1 = signal.savgol_filter(corrected_ppg_1, window_length=51, polyorder=3)

    # Find peaks and troughs-------------------------------------------------------------------------------
    peaks, _ = find_peaks(smoothed_ppg_1)  # Adjust height threshold as needed
    troughs, _ = find_peaks(-smoothed_ppg_1)  # Detect troughs by inverting signal

    # Calculate Pulse Amplitude
    if len(peaks) > 0 and len(troughs) > 0:
        pulse_amplitude = smoothed_ppg_1[peaks].mean() - smoothed_ppg_1[troughs].mean()
    else:
        pulse_amplitude = np.nan

    # Calculate Baseline Amplitude
    baseline_amplitude = smoothed_ppg_1.mean()

    # Calculate Perfusion Index
    if not np.isnan(pulse_amplitude) and not np.isnan(baseline_amplitude):
        perfusion_index = (pulse_amplitude / baseline_amplitude) * 100
    else:
        perfusion_index = np.nan

    # Add to avg
    avgPerfusion += perfusion_index

    # Print the AC Component for trial
    print(f'Perfusion Index of {file} : {perfusion_index}')

print('----------------------------------------------------------------------------------')
print(f'Average Perfusion Index of Trials using HighPass is : {avgPerfusion / numOfFiles}')

HIGHPASS

Perfusion Index of ./ppg_GREEN\green1_AR.csv : 538.605128061985
Perfusion Index of ./ppg_GREEN\green2_AR.csv : 373.0811133115464
Perfusion Index of ./ppg_GREEN\green3_AR.csv : 162.39730692861755
Perfusion Index of ./ppg_GREEN\green4_AR.csv : 530.005898154363
Perfusion Index of ./ppg_GREEN\green5_AR.csv : 285.95255519279965
---------------------------------------------------
Average Perfusion Index of Trials using HighPass is : 378.00840032986235


Convert Results to Readable Format